In [1]:
import os
import glob
import pandas as pd
import numpy as np 
import csv
import re
import string
import time

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import csv
import re
import string
import emoji

import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import warnings
warnings.filterwarnings("ignore")

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english')) - set(['at', 'do', 'your', 'from', 'to', 'out', 'no', 'the'])
#demoji.download_codes()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
#start = time.time()
indir = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200916-20200923/h01-20200916"
outfile = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200916-20200923/(C&A)September16th.csv"

#data = pd.read_csv(indir, low_memory=False)
#end = time.time()
#print("Read csv with pandas: ",(end-start),"sec")


In [4]:
def Data_Cleaning(data):
    
    cols = [0, 1, 3, 4, 5, 6, 7, 8, 9,
            10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
            22, 29,
            30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
            40, 41, 42, 43, 45, 
            50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
            60, 61, 62, 63, 64, 65, 67,
            72, 73, 74, 75, 76, 77]
    data = data[data.columns.values[cols]]

    data_en = data[data['lang'] == 'en'].reset_index(drop=True)

    # RT flag

    data_en['RT_Flag'] = data_en['text'].str[:2]=='RT'

    # Date check

    #data_en['Date_Flag'] = data_en['created_at'].str[:10] == 'Tue Aug 18'

    #data_en = data_en[np.where((data_en['Date_Flag'] == True),True,False)].reset_index(drop=True)

    data_en['extended_tweet_full_text'] = np.where(data_en['truncated'] == False, data_en['text'], data_en['extended_tweet_full_text'])
    
    data_en['RT_full_text'] = np.where(data_en['RT_full_text'].isna(),data_en['RT_text'],data_en['RT_full_text'])
    
    data_en['QT_full_text'] = np.where(data_en['QT_full_text'].isna(),data_en['QT_text'],data_en['QT_full_text'])
    
    data_en = data_en.drop_duplicates()
    
    return data_en

In [5]:
def Analysis(cleaned_data):
    
    #cleaned_data = non_rep.copy()

    cleaned_data['extended_tweet_full_text_duplicate'] = cleaned_data['extended_tweet_full_text']
    cleaned_data['RT_full_text_duplicate'] = cleaned_data['RT_full_text']
    cleaned_data['QT_full_text_duplicate'] = cleaned_data['QT_full_text']
    
    #Convert to lower case
    cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.lower()
    cleaned_data['RT_full_text'] = cleaned_data['RT_full_text'].str.lower()
    cleaned_data['QT_full_text'] = cleaned_data['QT_full_text'].str.lower()
    
    #Removing emojis
    def demoji(text):
        emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            u"\U00010000-\U0010ffff"
                                   "]+", flags=re.UNICODE)
        return(emoji_pattern.sub(r'', text))

    cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].astype(str)
    cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].apply(lambda x:demoji(x))

    cleaned_data['RT_full_text'] = cleaned_data['RT_full_text'].astype(str)
    cleaned_data['RT_full_text'] = cleaned_data['RT_full_text'].apply(lambda x:demoji(x))

    cleaned_data['QT_full_text'] = cleaned_data['QT_full_text'].astype(str)
    cleaned_data['QT_full_text'] = cleaned_data['QT_full_text'].apply(lambda x:demoji(x))
    
    #Remove URLs
    cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.replace(r"http\S+| www\S+| https\S+| \S+\.com\S+| \S+\.com", "", regex=True)

    cleaned_data['RT_full_text'] = cleaned_data['RT_full_text'].str.replace(r"http\S+| www\S+| https\S+| \S+\.com\S+| \S+\.com", "", regex=True)

    cleaned_data['QT_full_text'] = cleaned_data['QT_full_text'].str.replace(r"http\S+| www\S+| https\S+| \S+\.com\S+| \S+\.com", "", regex=True)
    
    #Remove user @
    cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.replace(r'\@[\w]+', "", regex=True)

    cleaned_data['RT_full_text'] = cleaned_data['RT_full_text'].str.replace(r'\@[\w]+', "", regex=True)

    cleaned_data['QT_full_text'] = cleaned_data['QT_full_text'].str.replace(r'\@[\w]+', "", regex=True)
    
    #Remove punctuations
    cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.translate(str.maketrans("", "", string.punctuation))

    cleaned_data['RT_full_text'] = cleaned_data['RT_full_text'].str.translate(str.maketrans("", "", string.punctuation))

    cleaned_data['QT_full_text'] = cleaned_data['QT_full_text'].str.translate(str.maketrans("", "", string.punctuation))
    
    #More Cleaning
    cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '', regex=True).str.replace('\n',' ', regex=True).str.replace('—',' ', regex=True).str.strip('“').str.strip('”').str.strip('’').str.lstrip(' ').str.rstrip(' ')
    cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.lower()\
              .str.replace('(@[a-z0-9]+)\w+',' ')\
              .str.replace('(http\S+)', ' ')\
              .str.replace('([^0-9a-z \t])',' ')\
              .str.replace(' +',' ')

    cleaned_data['RT_full_text'] = cleaned_data['RT_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '', regex=True).str.replace('\n',' ', regex=True).str.replace('—',' ', regex=True).str.strip('“').str.strip('”').str.strip('’').str.lstrip(' ').str.rstrip(' ')
    cleaned_data['RT_full_text'] = cleaned_data['RT_full_text'].str.lower()\
              .str.replace('(@[a-z0-9]+)\w+',' ')\
              .str.replace('(http\S+)', ' ')\
              .str.replace('([^0-9a-z \t])',' ')\
              .str.replace(' +',' ')

    cleaned_data['QT_full_text'] = cleaned_data['QT_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '', regex=True).str.replace('\n',' ', regex=True).str.replace('—',' ', regex=True).str.strip('“').str.strip('”').str.strip('’').str.lstrip(' ').str.rstrip(' ')
    cleaned_data['QT_full_text'] = cleaned_data['QT_full_text'].str.lower()\
              .str.replace('(@[a-z0-9]+)\w+',' ')\
              .str.replace('(http\S+)', ' ')\
              .str.replace('([^0-9a-z \t])',' ')\
              .str.replace(' +',' ')
    
    #Tokenizing
    cleaned_data.extended_tweet_full_text = cleaned_data.extended_tweet_full_text.astype(str)
    cleaned_data['extended_tweet_full_text'] = cleaned_data.apply(lambda row: nltk.word_tokenize(row.extended_tweet_full_text), axis=1)


    cleaned_data.RT_full_text = cleaned_data.RT_full_text.astype(str)
    cleaned_data['RT_full_text'] = cleaned_data.apply(lambda row: nltk.word_tokenize(row.RT_full_text), axis=1)


    cleaned_data.QT_full_text = cleaned_data.QT_full_text.astype(str)
    cleaned_data['QT_full_text'] = cleaned_data.apply(lambda row: nltk.word_tokenize(row.QT_full_text), axis=1)

    # remove stopwords
    cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].apply(lambda x: [word for word in x if word not in stop_words])

    cleaned_data['RT_full_text'] = cleaned_data['RT_full_text'].apply(lambda x: [word for word in x if word not in stop_words])

    cleaned_data['QT_full_text'] = cleaned_data['QT_full_text'].apply(lambda x: [word for word in x if word not in stop_words])

    #Stemming
    #ps = PorterStemmer()
    #cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].apply(lambda x: [ps.stem(y) for y in x])
    #cleaned_data['RT_full_text'] = cleaned_data['RT_full_text'].apply(lambda x: [ps.stem(y) for y in x])
    #cleaned_data['QT_full_text'] = cleaned_data['QT_full_text'].apply(lambda x: [ps.stem(y) for y in x])
   
    #POSTags     
    def get_wordnet_pos(word):
        """Map POS tag to first character lemmatize() accepts"""
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)

    #Lemmatizing
    lemmatizer = WordNetLemmatizer()
    cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].apply(lambda x: [lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in x])

    cleaned_data['RT_full_text'] = cleaned_data['RT_full_text'].apply(lambda x: [lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in x])

    cleaned_data['QT_full_text'] = cleaned_data['QT_full_text'].apply(lambda x: [lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in x])
    
    #Joining the lemmetized tokens to form string
    cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].apply(lambda x: " ".join([word for word in x]))

    cleaned_data['RT_full_text'] = cleaned_data['RT_full_text'].apply(lambda x: " ".join([word for word in x]))
    
    cleaned_data['QT_full_text'] = cleaned_data['QT_full_text'].apply(lambda x: " ".join([word for word in x]))
    
    #Remove punctuations
    cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.translate(str.maketrans("", "", string.punctuation))
    cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.replace("’", '').str.replace("“", '').str.replace("”", '')
    cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '', regex=True).str.replace('\n',' ', regex=True).str.replace('—',' ', regex=True).str.strip('“').str.strip('”').str.strip('’').str.lstrip(' ').str.rstrip(' ')

    cleaned_data['RT_full_text'] = cleaned_data['RT_full_text'].str.translate(str.maketrans("", "", string.punctuation))
    cleaned_data['RT_full_text'] = cleaned_data['RT_full_text'].str.replace("’", '').str.replace("“", '').str.replace("”", '')
    cleaned_data['RT_full_text'] = cleaned_data['RT_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '', regex=True).str.replace('\n',' ', regex=True).str.replace('—',' ', regex=True).str.strip('“').str.strip('”').str.strip('’').str.lstrip(' ').str.rstrip(' ')

    cleaned_data['QT_full_text'] = cleaned_data['QT_full_text'].str.translate(str.maketrans("", "", string.punctuation))
    cleaned_data['QT_full_text'] = cleaned_data['QT_full_text'].str.replace("’", '').str.replace("“", '').str.replace("”", '')
    cleaned_data['QT_full_text'] = cleaned_data['QT_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '', regex=True).str.replace('\n',' ', regex=True).str.replace('—',' ', regex=True).str.strip('“').str.strip('”').str.strip('’').str.lstrip(' ').str.rstrip(' ')
    
    return cleaned_data

In [10]:
def concatenate(indir, outfile):
    os.chdir(indir)
    fileList = glob.glob("*.csv")
    dfList = []
    for filename in fileList:
        print(filename)
        df = pd.read_csv(filename, header = 0, low_memory=False)
        cleaned_df = Data_Cleaning(df)
        final_df = Analysis(cleaned_df)
        dfList.append(final_df)
        print(len(dfList))
    print(len(dfList))
    concatDf = pd.concat(dfList, axis = 0)
    concatDf.to_csv(outfile, index = None)

In [7]:
start_1 = time.time()

concatenate(indir, outfile)

end_1= time.time()

h01-20200916-000151.csv
1
h01-20200916-000735.csv
2
h01-20200916-001620.csv
3
h01-20200916-002520.csv
4
h01-20200916-003429.csv
5
h01-20200916-004408.csv
6
h01-20200916-005412.csv
7
h01-20200916-010457.csv
8
h01-20200916-012737.csv
9
h01-20200916-015120.csv
10
h01-20200916-021458.csv
11
h01-20200916-023736.csv
12
h01-20200916-030011.csv
13
h01-20200916-032330.csv
14
h01-20200916-034721.csv
15
h01-20200916-041101.csv
16
h01-20200916-043450.csv
17
h01-20200916-045822.csv
18
h01-20200916-052035.csv
19
h01-20200916-054314.csv
20
h01-20200916-060453.csv
21
h01-20200916-062559.csv
22
22


In [8]:
print("Total time : ",(end_1 - start_1),"sec")

Total time :  697.2845194339752 sec


# 2nd file(17 September)

In [9]:
indir_2 = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200916-20200923/h01-20200917"
outfile_2 = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200916-20200923/(C&A)September17th.csv"

In [10]:
def concatenate(indir_2, outfile_2):
    os.chdir(indir_2)
    fileList = glob.glob("*.csv")
    dfList = []
    for filename in fileList:
        print(filename)
        df = pd.read_csv(filename, header = 0, low_memory=False)
        cleaned_df = Data_Cleaning(df)
        final_df = Analysis(cleaned_df)
        dfList.append(final_df)
        print(len(dfList))
    print(len(dfList))
    concatDf = pd.concat(dfList, axis = 0)
    concatDf.to_csv(outfile_2, index = None)

In [11]:
start_2 = time.time()

concatenate(indir_2, outfile_2)

end_2 = time.time()

h01-20200917-051048.csv
1
h01-20200917-051615.csv
2
h01-20200917-052717.csv
3
h01-20200917-053241.csv
4
h01-20200917-053803.csv
5
h01-20200917-054318.csv
6
h01-20200917-054830.csv
7
h01-20200917-055344.csv
8
h01-20200917-055905.csv
9
h01-20200917-060354.csv
10
h01-20200917-060903.csv
11
h01-20200917-061410.csv
12
h01-20200917-061912.csv
13
h01-20200917-062404.csv
14
h01-20200917-062846.csv
15
h01-20200917-063817.csv
16
h01-20200917-064253.csv
17
h01-20200917-064726.csv
18
h01-20200917-065200.csv
19
h01-20200917-065625.csv
20
h01-20200917-070040.csv
21
h01-20200917-070449.csv
22
h01-20200917-070942.csv
23
h01-20200917-191352.csv
24
h01-20200917-191743.csv
25
h01-20200917-192134.csv
26
h01-20200917-192524.csv
27
h01-20200917-192913.csv
28
h01-20200917-193259.csv
29
h01-20200917-193653.csv
30
h01-20200917-194450.csv
31
h01-20200917-194849.csv
32
h01-20200917-195254.csv
33
h01-20200917-195658.csv
34
h01-20200917-200053.csv
35
h01-20200917-200503.csv
36
h01-20200917-200914.csv
37
h01-202009

In [12]:
print("Total time : ",(end_2 - start_2),"sec")

Total time :  3561.8854143619537 sec


# 3rd file(18 September)

In [6]:
indir_3 = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200916-20200923/h01-20200918"
outfile_3 = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200916-20200923/(C&A)September18th.csv"

In [7]:
def concatenate(indir_3, outfile_3):
    os.chdir(indir_3)
    fileList = glob.glob("*.csv")
    dfList = []
    for filename in fileList:
        print(filename)
        df = pd.read_csv(filename, header = 0, low_memory=False)
        cleaned_df = Data_Cleaning(df)
        final_df = Analysis(cleaned_df)
        dfList.append(final_df)
        print(len(dfList))
    print(len(dfList))
    concatDf = pd.concat(dfList, axis = 0)
    concatDf.to_csv(outfile_3, index = None)

In [8]:
start_3 = time.time()

concatenate(indir_3, outfile_3)

end_3 = time.time()

h01-20200918-000407.csv
1
h01-20200918-000930.csv
2
h01-20200918-001451.csv
3
h01-20200918-002021.csv
4
h01-20200918-002558.csv
5
h01-20200918-003136.csv
6
h01-20200918-003725.csv
7
h01-20200918-004312.csv
8
h01-20200918-004911.csv
9
h01-20200918-005514.csv
10
h01-20200918-010121.csv
11
h01-20200918-010736.csv
12
h01-20200918-011353.csv
13
h01-20200918-012008.csv
14
h01-20200918-012632.csv
15
h01-20200918-013917.csv
16
h01-20200918-014554.csv
17
h01-20200918-015231.csv
18
h01-20200918-015910.csv
19
h01-20200918-020520.csv
20
h01-20200918-021128.csv
21
h01-20200918-021740.csv
22
h01-20200918-022358.csv
23
h01-20200918-023013.csv
24
h01-20200918-023631.csv
25
h01-20200918-024252.csv
26
h01-20200918-024913.csv
27
h01-20200918-025534.csv
28
h01-20200918-030137.csv
29
h01-20200918-030731.csv
30
h01-20200918-031932.csv
31
h01-20200918-032539.csv
32
h01-20200918-033151.csv
33
h01-20200918-033806.csv
34
h01-20200918-034419.csv
35
h01-20200918-035029.csv
36
h01-20200918-035644.csv
37
h01-202009

In [9]:
print("Total time : ",(end_3 - start_3),"sec")

Total time :  12760.105634450912 sec


# 4th file(19 September)

In [11]:
indir_4 = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200916-20200923/h01-20200919"
outfile_4 = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200916-20200923/(C&A)September19th.csv"

In [12]:
def concatenate(indir_4, outfile_4):
    os.chdir(indir_4)
    fileList = glob.glob("*.csv")
    dfList = []
    for filename in fileList:
        print(filename)
        df = pd.read_csv(filename, header = 0, low_memory=False)
        cleaned_df = Data_Cleaning(df)
        final_df = Analysis(cleaned_df)
        dfList.append(final_df)
        print(len(dfList))
    print(len(dfList))
    concatDf = pd.concat(dfList, axis = 0)
    concatDf.to_csv(outfile_4, index = None)

In [13]:
start_4 = time.time()

concatenate(indir_4, outfile_4)

end_4 = time.time()

h01-20200919-000308.csv
1
h01-20200919-001027.csv
2
h01-20200919-001750.csv
3
h01-20200919-002528.csv
4
h01-20200919-003307.csv
5
h01-20200919-004050.csv
6
h01-20200919-004847.csv
7
h01-20200919-005703.csv
8
h01-20200919-010520.csv
9
h01-20200919-011353.csv
10
h01-20200919-012240.csv
11
h01-20200919-013136.csv
12
h01-20200919-014029.csv
13
h01-20200919-014930.csv
14
h01-20200919-015829.csv
15
h01-20200919-021502.csv
16
h01-20200919-022306.csv
17
h01-20200919-023059.csv
18
h01-20200919-023854.csv
19
h01-20200919-024643.csv
20
h01-20200919-025406.csv
21
h01-20200919-030129.csv
22
h01-20200919-030839.csv
23
h01-20200919-031604.csv
24
h01-20200919-032325.csv
25
h01-20200919-033038.csv
26
h01-20200919-033801.csv
27
h01-20200919-034524.csv
28
h01-20200919-035507.csv
29
h01-20200919-040440.csv
30
h01-20200919-042415.csv
31
h01-20200919-043358.csv
32
h01-20200919-044332.csv
33
h01-20200919-045304.csv
34
h01-20200919-050225.csv
35
h01-20200919-051155.csv
36
h01-20200919-052120.csv
37
h01-202009

In [14]:
print("Total time : ",(end_4 - start_4),"sec")

Total time :  8019.031992673874 sec


# 5th file(20 September)

In [15]:
indir_5 = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200916-20200923/h01-20200920"
outfile_5 = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200916-20200923/(C&A)September20th.csv"

In [16]:
def concatenate(indir_5, outfile_5):
    os.chdir(indir_5)
    fileList = glob.glob("*.csv")
    dfList = []
    for filename in fileList:
        print(filename)
        df = pd.read_csv(filename, header = 0, low_memory=False)
        cleaned_df = Data_Cleaning(df)
        final_df = Analysis(cleaned_df)
        dfList.append(final_df)
        print(len(dfList))
    print(len(dfList))
    concatDf = pd.concat(dfList, axis = 0)
    concatDf.to_csv(outfile_5, index = None)

In [17]:
start_5 = time.time()

concatenate(indir_5, outfile_5)

end_5 = time.time()

h01-20200920-000658.csv
1
h01-20200920-002157.csv
2
h01-20200920-003742.csv
3
h01-20200920-005355.csv
4
h01-20200920-011121.csv
5
h01-20200920-012846.csv
6
h01-20200920-014632.csv
7
h01-20200920-020417.csv
8
h01-20200920-022125.csv
9
h01-20200920-023817.csv
10
h01-20200920-025446.csv
11
h01-20200920-031043.csv
12
h01-20200920-032609.csv
13
h01-20200920-034118.csv
14
h01-20200920-035634.csv
15
h01-20200920-042552.csv
16
h01-20200920-044034.csv
17
h01-20200920-045459.csv
18
h01-20200920-050835.csv
19
h01-20200920-052228.csv
20
h01-20200920-053618.csv
21
h01-20200920-054955.csv
22
h01-20200920-060318.csv
23
h01-20200920-061631.csv
24
h01-20200920-063018.csv
25
h01-20200920-064310.csv
26
h01-20200920-065608.csv
27
h01-20200920-070842.csv
28
h01-20200920-072100.csv
29
h01-20200920-073322.csv
30
h01-20200920-075728.csv
31
h01-20200920-080909.csv
32
h01-20200920-082025.csv
33
h01-20200920-082651.csv
34
h01-20200920-083244.csv
35
h01-20200920-085415.csv
36
h01-20200920-091451.csv
37
h01-202009

In [18]:
print("Total time : ",(end_5 - start_5),"sec")

Total time :  8141.390430212021 sec


# 6th file(21 September)

In [19]:
indir_6 = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200916-20200923/h01-20200921"
outfile_6 = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200916-20200923/(C&A)September21st.csv"

In [20]:
def concatenate(indir_6, outfile_6):
    os.chdir(indir_6)
    fileList = glob.glob("*.csv")
    dfList = []
    for filename in fileList:
        print(filename)
        df = pd.read_csv(filename, header = 0, low_memory=False)
        cleaned_df = Data_Cleaning(df)
        final_df = Analysis(cleaned_df)
        dfList.append(final_df)
        print(len(dfList))
    print(len(dfList))
    concatDf = pd.concat(dfList, axis = 0)
    concatDf.to_csv(outfile_6, index = None)

In [21]:
start_6 = time.time()

concatenate(indir_6, outfile_6)

end_6 = time.time()

h01-20200921-002132.csv
1
h01-20200921-002930.csv
2
h01-20200921-003602.csv
3
h01-20200921-004237.csv
4
h01-20200921-004907.csv
5
h01-20200921-005535.csv
6
h01-20200921-010147.csv
7
h01-20200921-010807.csv
8
h01-20200921-011406.csv
9
h01-20200921-012343.csv
10
h01-20200921-012930.csv
11
h01-20200921-013516.csv
12
h01-20200921-014108.csv
13
h01-20200921-014655.csv
14
h01-20200921-015244.csv
15
h01-20200921-021606.csv
16
h01-20200921-022707.csv
17
h01-20200921-023300.csv
18
h01-20200921-023904.csv
19
h01-20200921-024509.csv
20
h01-20200921-025115.csv
21
h01-20200921-025726.csv
22
h01-20200921-030328.csv
23
h01-20200921-030938.csv
24
h01-20200921-031551.csv
25
h01-20200921-032204.csv
26
h01-20200921-032813.csv
27
h01-20200921-033415.csv
28
h01-20200921-034021.csv
29
h01-20200921-034633.csv
30
h01-20200921-035848.csv
31
h01-20200921-040437.csv
32
h01-20200921-041043.csv
33
h01-20200921-041640.csv
34
h01-20200921-042246.csv
35
h01-20200921-042851.csv
36
h01-20200921-043453.csv
37
h01-202009

In [22]:
print("Total time : ",(end_6 - start_6),"sec")

Total time :  9616.265283584595 sec


# 7th file(22 September)

In [6]:
indir_7 = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200916-20200923/h01-20200922"
outfile_7 = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200916-20200923/(C&A)September22nd.csv"

In [7]:
def concatenate(indir_7, outfile_7):
    os.chdir(indir_7)
    fileList = glob.glob("*.csv")
    dfList = []
    for filename in fileList:
        print(filename)
        df = pd.read_csv(filename, header = 0, low_memory=False)
        cleaned_df = Data_Cleaning(df)
        final_df = Analysis(cleaned_df)
        dfList.append(final_df)
        print(len(dfList))
    print(len(dfList))
    concatDf = pd.concat(dfList, axis = 0)
    concatDf.to_csv(outfile_7, index = None)

In [8]:
start_7 = time.time()

concatenate(indir_7, outfile_7)

end_7 = time.time()

h01-20200922-000416.csv
1
h01-20200922-000919.csv
2
h01-20200922-001425.csv
3
h01-20200922-001939.csv
4
h01-20200922-002459.csv
5
h01-20200922-003018.csv
6
h01-20200922-003537.csv
7
h01-20200922-004059.csv
8
h01-20200922-004630.csv
9
h01-20200922-005159.csv
10
h01-20200922-005734.csv
11
h01-20200922-010253.csv
12
h01-20200922-010822.csv
13
h01-20200922-011351.csv
14
h01-20200922-011921.csv
15
h01-20200922-013019.csv
16
h01-20200922-013600.csv
17
h01-20200922-014144.csv
18
h01-20200922-014727.csv
19
h01-20200922-015308.csv
20
h01-20200922-015852.csv
21
h01-20200922-020426.csv
22
h01-20200922-021009.csv
23
h01-20200922-021552.csv
24
h01-20200922-022128.csv
25
h01-20200922-022708.csv
26
h01-20200922-023245.csv
27
h01-20200922-023824.csv
28
h01-20200922-024403.csv
29
h01-20200922-024939.csv
30
h01-20200922-030052.csv
31
h01-20200922-030616.csv
32
h01-20200922-031144.csv
33
h01-20200922-031712.csv
34
h01-20200922-032240.csv
35
h01-20200922-032809.csv
36
h01-20200922-033337.csv
37
h01-202009

EmptyDataError: No columns to parse from file

In [9]:
print("Total time : ",(end_7 - start_7),"sec")

NameError: name 'end_7' is not defined

# 8th file(23 September)

In [10]:
indir_8 = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200916-20200923/h01-20200923"
outfile_8 = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200916-20200923/(C&A)September23rd.csv"

In [11]:
def concatenate(indir_8, outfile_8):
    os.chdir(indir_8)
    fileList = glob.glob("*.csv")
    dfList = []
    for filename in fileList:
        print(filename)
        df = pd.read_csv(filename, header = 0, low_memory=False)
        cleaned_df = Data_Cleaning(df)
        final_df = Analysis(cleaned_df)
        dfList.append(final_df)
        print(len(dfList))
    print(len(dfList))
    concatDf = pd.concat(dfList, axis = 0)
    concatDf.to_csv(outfile_8, index = None)

In [12]:
start_8 = time.time()

concatenate(indir_8, outfile_8)

end_8 = time.time()

h01-20200923-000517.csv
1
h01-20200923-001112.csv
2
h01-20200923-001715.csv
3
h01-20200923-002527.csv
4
h01-20200923-003348.csv
5
h01-20200923-004233.csv
6
h01-20200923-005112.csv
7
h01-20200923-010014.csv
8
h01-20200923-010851.csv
9
h01-20200923-011749.csv
10
h01-20200923-012649.csv
11
h01-20200923-013602.csv
12
h01-20200923-014511.csv
13
h01-20200923-015418.csv
14
h01-20200923-020109.csv
15
h01-20200923-021418.csv
16
h01-20200923-022049.csv
17
h01-20200923-022721.csv
18
h01-20200923-023343.csv
19
h01-20200923-024008.csv
20
h01-20200923-024631.csv
21
h01-20200923-025513.csv
22
h01-20200923-030726.csv
23
h01-20200923-031934.csv
24
h01-20200923-033142.csv
25
h01-20200923-034407.csv
26
h01-20200923-035658.csv
27
h01-20200923-040936.csv
28
h01-20200923-042211.csv
29
h01-20200923-043432.csv
30
h01-20200923-051056.csv
31
h01-20200923-053449.csv
32
h01-20200923-055716.csv
33
h01-20200923-061853.csv
34
h01-20200923-062627.csv
35
h01-20200923-063149.csv
36
h01-20200923-063713.csv
37
h01-202009

In [13]:
print("Total time : ",(end_8 - start_8),"sec")

Total time :  12287.601283550262 sec


# Total time

In [15]:
indir_9 = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200924-20200930/h01-20200924"
outfile_9 = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200924-20200930/(C&A)September24th.csv"

In [16]:
def concatenate(indir_9, outfile_9):
    os.chdir(indir_9)
    fileList = glob.glob("*.csv")
    dfList = []
    for filename in fileList:
        print(filename)
        df = pd.read_csv(filename, header = 0, low_memory=False)
        cleaned_df = Data_Cleaning(df)
        final_df = Analysis(cleaned_df)
        dfList.append(final_df)
        print(len(dfList))
    print(len(dfList))
    concatDf = pd.concat(dfList, axis = 0)
    concatDf.to_csv(outfile_9, index = None)

In [17]:
start_9 = time.time()

concatenate(indir_9, outfile_9)

end_9 = time.time()

h01-20200924-032626.csv
1
h01-20200924-033231.csv
2
h01-20200924-033837.csv
3
h01-20200924-034438.csv
4
h01-20200924-035040.csv
5
h01-20200924-035652.csv
6
h01-20200924-040243.csv
7
h01-20200924-040853.csv
8
h01-20200924-041503.csv
9
h01-20200924-042113.csv
10
h01-20200924-042724.csv
11
h01-20200924-043316.csv
12
h01-20200924-043911.csv
13
h01-20200924-044511.csv
14
h01-20200924-045100.csv
15
h01-20200924-050229.csv
16
h01-20200924-050820.csv
17
h01-20200924-051412.csv
18
h01-20200924-051957.csv
19
h01-20200924-052540.csv
20
h01-20200924-053114.csv
21
h01-20200924-053656.csv
22
h01-20200924-054236.csv
23
h01-20200924-054816.csv
24
h01-20200924-055546.csv
25
h01-20200924-060251.csv
26
h01-20200924-061012.csv
27
h01-20200924-061710.csv
28
h01-20200924-062405.csv
29
h01-20200924-063048.csv
30
h01-20200924-064406.csv
31
h01-20200924-065035.csv
32
h01-20200924-065702.csv
33
h01-20200924-070308.csv
34
h01-20200924-070915.csv
35
h01-20200924-071514.csv
36
h01-20200924-072113.csv
37
h01-202009

In [18]:
print("Total time : ",(end_9 - start_9),"sec")

Total time :  8763.09886097908 sec


In [19]:
indir_10 = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200924-20200930/h01-20200925"
outfile_10 = "/media/manikya_varshney/My Passport/Mani/Mani(Yale)/Data/h01-20200924-20200930/(C&A)September25th.csv"

In [20]:
def concatenate(indir_10, outfile_10):
    os.chdir(indir_10)
    fileList = glob.glob("*.csv")
    dfList = []
    for filename in fileList:
        print(filename)
        df = pd.read_csv(filename, header = 0, low_memory=False)
        cleaned_df = Data_Cleaning(df)
        final_df = Analysis(cleaned_df)
        dfList.append(final_df)
        print(len(dfList))
    print(len(dfList))
    concatDf = pd.concat(dfList, axis = 0)
    concatDf.to_csv(outfile_10, index = None)

In [21]:
start_10 = time.time()

concatenate(indir_10, outfile_10)

end_10 = time.time()

h01-20200925-000005.csv
1
h01-20200925-001252.csv
2
h01-20200925-002611.csv
3
h01-20200925-005337.csv
4
h01-20200925-012142.csv
5
h01-20200925-014536.csv
6
h01-20200925-015303.csv
7
h01-20200925-020013.csv
8
h01-20200925-020703.csv
9
h01-20200925-021405.csv
10
h01-20200925-022325.csv
11
h01-20200925-023239.csv
12
h01-20200925-024226.csv
13
h01-20200925-025616.csv
14
h01-20200925-030939.csv
15
h01-20200925-033519.csv
16
h01-20200925-034658.csv
17
h01-20200925-035840.csv
18
h01-20200925-041005.csv
19
h01-20200925-042159.csv
20
h01-20200925-043407.csv
21
h01-20200925-044630.csv
22
h01-20200925-045846.csv
23
h01-20200925-051034.csv
24
h01-20200925-052235.csv
25
h01-20200925-053432.csv
26
h01-20200925-054631.csv
27
h01-20200925-055809.csv
28
h01-20200925-060944.csv
29
h01-20200925-061544.csv
30
h01-20200925-062652.csv
31
h01-20200925-063215.csv
32
h01-20200925-063744.csv
33
h01-20200925-064311.csv
34
h01-20200925-064834.csv
35
h01-20200925-065356.csv
36
h01-20200925-065916.csv
37
h01-202009

In [22]:
print("Total time : ",(end_10 - start_10),"sec")

Total time :  9049.049993753433 sec
